# VOLTANDO UMA PASTA PARA TRAZ

In [ ]:
import os
os.chdir('../')

# IMPORTS

In [ ]:
from modules.youtube.extract.ytbpy import channel, playlist, utils, video
from modules import db_manager
from datetime import datetime
from jinja2 import Template
from tqdm import tqdm
import pandas as pd
import json
import yt_dlp
import time
import os

path_read1          = "repository\\querys\\operation\\read\\game_event.sql"
path_update         = "repository\\querys\\operation\\insert\\game_event.sql"

def wait(seconds=60):
    for _ in range(seconds):
        time.sleep(1)

# lendo tabela de eventos 
with open(path_read1, "r", encoding="utf-8") as file:
    query_read1     = file.read()

# lendo tabela de eventos 
with open(path_update, "r", encoding="utf-8") as file:
    query_update1   = file.read()

# ESCREVENTOS OS EVENTOS NO BANCO DE DADOS

In [ ]:
import json

df_events           = db_manager.read_df(query_read1)

def ler_json_em_stream(caminho_arquivo):
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        for linha in f:
            try:
                yield json.loads(linha)
            except json.JSONDecodeError:
                continue
    return None

# Exemplo de uso:
events = []
for item in ler_json_em_stream('data\\temp\\saida.json'):
    if item['type'] not in events:
        
        sql_template = Template(query_update1)
        rendered_sql = sql_template.render(
            ID_GAME = 1, # ID DO DOTA 2
            NAME    = item['type']
        ).replace("'None'", "NULL").replace("None", "NULL")\
        .replace("'False'", "FALSE").replace("False", "FALSE")\
        .replace("'True'", "TRUE").replace("True", "TRUE")
        
        try:
            db_manager.execute_raw(rendered_sql)
        except Exception as e:
            print(f"Error: {e}")
            db_manager.connection.rollback()
            
        events.append(item['type'])

# GERANDO TEMPO DE RECORTES 

In [ ]:
# Exemplo de uso:
structured_data     = {
    'time_Start': None,
    'steamids': {},
    'player_slot': {}
}
for item in ler_json_em_stream('data\\temp\\saida.json'):
    if item['type'] == 'draft_start':
        structured_data['time_Start'] = item['time']
        continue
    if item['type']: # SE JOGO TIVER TEMPO DIFERENTE DE NONE O GAME FOI INICIADO
        if item['type'] == 'steamid':
            structured_data['steamids'][item['key']] = item['unit']
            continue
        if item['type'] == 'player_slot':
            structured_data['player_slot'][item['value']] = item['key']